In [1]:
# !pip install PyTorchCML

In [1]:
import sys
sys.path.append("../../")

from itertools import product

from PyTorchCML import losses, models, samplers, regularizers, evaluators, trainers
import torch
from torch import nn, optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

In [2]:
# download movielens dataset
movielens = pd.read_csv(
  'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 
    sep='\t', header=None, index_col=None,
    names = ["user_id", "item_id", "rating", "timestamp"]
)
# Set user/item id and number of users/items.
movielens.user_id -= 1
movielens.item_id -= 1
n_user = movielens.user_id.nunique()
n_item = movielens.item_id.nunique()

# make implicit feedback
movielens.rating = (movielens.rating >= 4).astype(int)


# train test split
train, test = train_test_split(movielens)


# all user item pairs
df_all = pd.DataFrame(
    [[u, i] for u,i in product(range(n_user), range(n_item))],
    columns=["user_id", "item_id"]
)

# frag train pairs
df_all = pd.merge(
    df_all, 
    train[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
)

# remove train pairs
test = pd.merge(
    df_all[df_all.rating.isna()][["user_id", "item_id"]], 
    test[["user_id", "item_id", "rating"]], 
    on=["user_id", "item_id"], 
    how="left"
).fillna(0)

# numpy array
train_set = train[train.rating == 1][["user_id", "item_id"]].values
test_set = test[["user_id", "item_id", "rating"]].values

# to torch.Tensor
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)


## Defalt

In [3]:
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks=[3,5])
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler)


In [4]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:21<00:00, 43.05it/s]


In [5]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.007423,0.012902,0.001568,0.008634,0.017922,0.002917,0,NaN
0,0.042387,0.070078,0.006008,0.046897,0.084353,0.011537,10,0.432954
0,0.202032,0.291888,0.044877,0.202131,0.311188,0.073077,20,0.274121


## Strict Negative

In [6]:
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=True)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks=[3,5])
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler)


In [7]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:24<00:00, 39.21it/s]


In [8]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.020385,0.036939,0.001419,0.018477,0.041640,0.002321,0,NaN
0,0.059936,0.091552,0.004546,0.068632,0.110379,0.009836,10,0.471340
0,0.273629,0.369123,0.034714,0.272501,0.385817,0.059150,20,0.287908


## Global Orthogonal Regularization

In [9]:
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
regs = [regularizers.GlobalOrthogonalRegularizer(weight=1e-2)]
criterion = losses.SumTripletLoss(margin=1, regularizers=regs).to(device)
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=True)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks=[3,5])
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler)

In [10]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:24<00:00, 38.13it/s]


In [11]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.017032,0.029692,0.002178,0.017122,0.036462,0.003362,0,NaN
0,0.074296,0.115500,0.005293,0.075128,0.125672,0.008741,10,0.469893
0,0.276036,0.380877,0.035444,0.276884,0.395498,0.060833,20,0.281246


## Two Stage

In [3]:
item_count = train.groupby("item_id")["user_id"].count()
count_index = np.array(item_count.index)
neg_weight = np.zeros(n_item)
neg_weight[count_index] = item_count ** 0.1

In [4]:
lr = 1e-3
n_dim = 10
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

regs = [regularizers.GlobalOrthogonalRegularizer(weight=1e-3)]
criterion = losses.MinTripletLoss(margin=1, regularizers=regs).to(device)
sampler = samplers.TwoStageSampler(
    train_set, n_user, n_item, 
    neg_weight=neg_weight, n_neg_samples=5,
    device=device, strict_negative=False
)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks=[3,5])
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler)

In [5]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:22<00:00, 41.77it/s]


In [6]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.012986,0.022004,0.001167,0.013278,0.028190,0.002582,0,NaN
0,0.207166,0.293655,0.019897,0.200064,0.305768,0.033076,10,1.043901
0,0.356546,0.484093,0.052573,0.326033,0.484409,0.074481,20,1.001474


## model weighted negative sampler

In [4]:
def svd_init(X, dim):
    """
    Args :
        X : csr_matrix which element is 0 or 1.
        dim : number of dimention
    """
    svd = TruncatedSVD(n_components=10)
    U_ = svd.fit_transform(X)
    V_ = svd.components_

    s = (U_.sum(axis=1).mean() + V_.sum(axis=0).mean()) / 2
    U = 2 ** 0.5 * U_ - (1 / n_dim) ** 0.5 * s * np.ones_like(U_)
    V = 2 ** 0.5 * V_ + (1 / n_dim) ** 0.5 / s * np.ones_like(V_)
    ub = -(2 / n_dim) ** 0.5 * U_.sum(axis=1) / s
    vb = (2 / n_dim) ** 0.5 * V_.sum(axis=0) * s

    return U, V, ub, vb

In [5]:
n_dim = 10
X = csr_matrix(
    (np.ones(train_set.shape[0]), (train_set[:,0], train_set[:,1])),
    shape=[n_user, n_item]
)
U, V, ub, vb = svd_init(X, n_dim)
neg_weight_model = models.LogitMatrixFactorization(
    n_user, n_item, n_dim, max_norm=None,
    user_embedding_init = torch.Tensor(U), 
    item_embedding_init = torch.Tensor(V.T),
    user_bias_init = torch.Tensor(ub), 
    item_bias_init = torch.Tensor(vb)
).to(device)
neg_weight_model.link_weight = lambda x : 1 - torch.sigmoid(x)

In [6]:
lr = 1e-3
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = losses.SumTripletLoss(margin=1).to(device)
sampler = samplers.BaseSampler(
    train_set, n_user, n_item, 
    neg_weight=neg_weight_model,
    device=device, strict_negative=False
)

score_function_dict = {
    "nDCG" : evaluators.ndcg,
    "MAP" : evaluators.average_precision,
    "Recall": evaluators.recall
}
evaluator = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks=[3,5])
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler)

In [7]:
trainer.fit(n_batch=256, n_epoch=20, valid_evaluator = evaluator, valid_per_epoch=10)

100%|██████████| 943/943 [00:20<00:00, 45.11it/s]


In [8]:
trainer.valid_scores

,nDCG@3,MAP@3,Recall@3,nDCG@5,MAP@5,Recall@5,epoch,loss
0,0.016209,0.030399,0.001955,0.01690,0.038264,0.003477,0,NaN
0,0.051292,0.078208,0.004285,0.05536,0.094836,0.007510,10,0.572125
0,0.233268,0.322552,0.030232,0.23401,0.336276,0.049536,20,0.430135
